In [1]:
from langchain.llms.openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [2]:
def load_and_format_file(csv_file):
    '''
    - Takes in a csv file and creates file_content list object of the lines in the csv file
    - Returns a single string of concatenated elemnts in file_contents
    '''
    with open(csv_file, "r") as file:
        file_contents = file.readlines()
    return ''.join(file_contents)

In [3]:
def load_openai_llm(api_key, model="gpt-3.5-turbo-instruct-0914", temperature=0.4):
    '''
    - Creates OpenAI model
    '''
    llm = OpenAI(openai_api_key=api_key, model=model, temperature=temperature)
    return llm

In [4]:
def ask_ai(csv_file, api_key, task_number, original_budget, spent_budget, starting_date, deadline_date, date):
    '''
    - Takes in csv and farmats it into a single string
    - Creates an OpenAI model
    - Propts the model to return task status color and recommendations
    - Returns the task number, status color, and recommendation response
    '''
    formatted_file_contents = load_and_format_file(csv_file)
    llm = load_openai_llm(api_key)

    prompt = PromptTemplate(
        input_variables=["original_budget", "spent_budget", "starting_date", "deadline_date", "task_number", "date"],
        template=f"""You are an expert in project management. you are given this file which contains data of tasks in a project {formatted_file_contents} For task number {{task_number}}, the original budget allocated for the task is {{original_budget}}. From this amount we have so far spent {{spent_budget}}, the task starting date is {{starting_date}}, and task deadline is {{deadline_date}}. Today is {{date}}. Use your understanding of project management to return a RAG color representing the task status and a recommendation in this format [COLOR, Recommendation] for task {{task_number}}. Your recommendation must include 3 expert exdvices about three important aspects of current status of the task number {{task_number}}
        """
    )

    chain = LLMChain(llm=llm, prompt=prompt)

    response = chain.run({
        'original_budget' : original_budget,
        'spent_budget' : spent_budget,
        'starting_date' : starting_date,
        'deadline_date' : deadline_date,
        'task_number': task_number,
        'date' : date
    })

    status = response.split(",")[0].strip().lower()
    status = status.replace("[", "").replace("]", "").strip()

    return task_number, status, response

In [5]:
def update_status_and_budget_in_csv(task_number, new_status, spent_budget, csv_file, recommendation):
    '''
    - Takes in the csv file, task number, the spent budget, the new status and recommendation from the model
    - Locates the status datapoint of the selected task and updates the status
    - Adds the task recommendation in the recommendation column
    '''

    valid_statuses = ["r", "red", "g", "green", "a", "amber", "o", "orange"]

    if new_status not in valid_statuses:
        return

    with open(csv_file, "r") as file:
        rows = file.readlines()

    header = rows[0].strip().split(',')
    status_index = header.index('Status')
    budget_index = header.index('Spent budget')

    # Check if the recommendation column exists, if not, add it
    if 'Recommendation' not in header:
        header.append('Recommendation')
        rows[0] = ",".join(header) + "\n"

    recommendation_index = header.index('Recommendation')

    for index, row in enumerate(rows):
        row_elements = row.strip().split(',')
        if row_elements[0] == str(task_number):
            row_elements[status_index] = new_status
            row_elements[budget_index] = str(spent_budget)
            # Check if row already has recommendation column data, if not, add a placeholder
            if len(row_elements) <= recommendation_index:
                row_elements.append("")
            row_elements[recommendation_index] = recommendation
            rows[index] = ",".join(row_elements) + "\n"

    with open(csv_file, "w") as file:
        file.writelines(rows)

In [6]:
#(csv_file, api_key, task_number, original_budget, spent_budget, starting_date, deadline_date, date)
task_number_1, status_1, response_1 = ask_ai("../tasks.csv", OPENAI_API_KEY, 1, 20000, 23600,"2023-10-01","2023-10-31", "2023-10-11")

In [7]:
task_number_1, status_1, response_1

(1,
 'red',
 '\n[red, "The current budget for task 1 has exceeded the allocated maximum budget. It is recommended to review the budget and make necessary adjustments to stay within the allocated maximum budget. Additionally, it is important to closely monitor the spending for this task and make any necessary adjustments to prevent further budget overruns. It is also recommended to communicate the budget issues to the project stakeholders and seek their support in managing the budget effectively."]')

In [8]:
#(csv_file, api_key, task_number, original_budget, spent_budget, starting_date, deadline_date, date)
task_number_2, status_2, response_2 = ask_ai("../tasks.csv", OPENAI_API_KEY, 2, 15000, 1200,"2023-10-05","2023-11-15", "2023-10-11")

In [9]:
task_number_2, status_2, response_2

(2,
 'green',
 '\n[green, "The current status of task 2 is on track and within the allocated budget. However, it is important to closely monitor the progress and ensure that the remaining budget is used efficiently. Additionally, regular communication with the team and stakeholders is crucial to keep everyone informed and address any potential issues in a timely manner. Lastly, it is recommended to regularly review the project plan and make necessary adjustments to ensure the task stays on track and is completed within the allocated time frame."]')

In [12]:
#(csv_file, api_key, task_number, original_budget, spent_budget, starting_date, deadline_date, date)
task_number_4, status_4, response_4 = ask_ai("../tasks.csv", OPENAI_API_KEY, 4, 18000, 19500,"2023-10-10","2023-12-01", "2023-10-11")

In [13]:
task_number_4, status_4, response_4

(4,
 'red',
 '\n[red, "The current budget for task 4 has exceeded the allocated maximum budget. It is recommended to review the budget and make necessary adjustments to stay within the allocated maximum budget. Additionally, it is important to closely monitor the spending and make sure it stays within the allocated budget. It is also recommended to reassess the timeline and make necessary adjustments to ensure the task is completed within the given deadline. Finally, it is important to communicate any budget or timeline changes to all stakeholders to ensure transparency and avoid any potential delays or issues."]')